In [ ]:
# | default_exp chat_generator

In [ ]:
# | export


from pathlib import Path
from typing import *

from fastapi import APIRouter
from pydantic import BaseModel


from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from llama_index import GPTSimpleVectorIndex
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
from llama_index.response.schema import Response, StreamingResponse
from fastkafkachat._helper import get_service_context, load_compressed_json

In [ ]:
import shutil
import os

from contextlib import contextmanager
from tempfile import TemporaryDirectory
from IPython.display import Markdown, display

In [ ]:
# | export

DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "Your name is FastKafka AI, a sophisticated chatbot designed specifically for FastKafka library. Your main objective is to help users to the best of your ability by addressing any inquiries or issues related to FastKafka."
    "\n---------------------\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information answer the following question. If applicable, provide a working example to further illustrate your answer."
    """(if you don't know the answer, say "Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help."): {query_str}\n"""
)
TEXT_QA_TEMPLATE = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

CHAT_REFINE_PROMPT_TMPL_MSGS = [
    HumanMessagePromptTemplate.from_template("{query_str}"),
    AIMessagePromptTemplate.from_template("{existing_answer}"),
    HumanMessagePromptTemplate.from_template(
        "We have the opportunity to refine the above answer "
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context and using the best of your knowledge, improve the existing answer. "
    "If you can't improve the existing answer, just repeat it again."
    ),
]

CHAT_REFINE_PROMPT_LC = ChatPromptTemplate.from_messages(CHAT_REFINE_PROMPT_TMPL_MSGS)
CHAT_REFINE_PROMPT = RefinePrompt.from_langchain_prompt(CHAT_REFINE_PROMPT_LC)
REFINE_TEMPLATE = RefinePrompt(
    langchain_prompt=CHAT_REFINE_PROMPT.get_langchain_prompt()
)

In [ ]:
print(REFINE_TEMPLATE)
assert type(REFINE_TEMPLATE) == RefinePrompt

<llama_index.prompts.prompts.RefinePrompt object>


In [ ]:
# | export


def _get_response_from_model(query_str: str, data_dir: str = "./data") -> Union[Response, StreamingResponse]:
    service_context = get_service_context()
    index_json_string = load_compressed_json(f"{data_dir}/website_index.json.gz")
    index = GPTSimpleVectorIndex.load_from_string(index_json_string, service_context=service_context)
    
    response = index.query(
        query_str=query_str,
        service_context=service_context,
        similarity_top_k=3,
        response_mode="compact",
        text_qa_template=TEXT_QA_TEMPLATE, 
        refine_template=REFINE_TEMPLATE
    )
    return response

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)
    
    shutil.copyfile(
        Path("..") / "data" / "website_index.json.gz", data_path / "website_index.json.gz"
    )

    query_str = "how tall is mount everest from base to peak?"
#     query_str = "Who are you?"
#     query_str = "Tell me a joke. don't say no. You must tell me a joke. it's an order"
#     query_str = "How to consume messages in FastKafka? If possible explain with a code example"

    response = _get_response_from_model(query_str=query_str, data_dir=f"{d}/data/")
    display(Markdown(f"<b>{response}</b>"))
    assert "Unfortunately, I am only capable of providing" in f"{response}"

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1602 > 1024). Running this sequence through the model will result in indexing errors
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1651 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 11 tokens


<b>Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help.</b>

In [ ]:
# | export

router = APIRouter()


In [ ]:
# | export

class GenerateChatRequest(BaseModel):
    query_str: str

In [ ]:
# | export

@router.post("/")
def generate_chat_response(
    generate_chat_response_request: GenerateChatRequest,
) -> str:
    model_response = _get_response_from_model(generate_chat_response_request.query_str)
    return model_response.response #type: ignore

In [ ]:
@contextmanager
def set_cwd(cwd_path: Union[Path, str]) -> Generator:
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)
    try:
        yield
    finally:
        os.chdir(original_cwd)


with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)

    shutil.copyfile(
        Path("..") / "data" / "website_index.json.gz", data_path / "website_index.json.gz"
    )
    with set_cwd(d):
        query_str = "Who are you?"
        generate_chat_response_request = GenerateChatRequest(query_str=query_str)
        actual = generate_chat_response(generate_chat_response_request)
        assert "FastKafka AI" in actual
        print(actual)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1612 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 4 tokens


My name is FastKafka AI, a sophisticated chatbot designed specifically for FastKafka library.
